<a href="https://colab.research.google.com/github/cherypallysaisurya/ResuVerse/blob/main/data_extrac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers python-docx PyPDF2 Pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.0 MB/s eta 0:00:00


**json code**

In [ ]:
!pip install spacy langchain
!python -m spacy download en_core_web_lg



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import os
import re
import json
import hashlib
from datetime import datetime
from docx import Document
from PyPDF2 import PdfReader
import spacy
from transformers import pipeline
from PIL import Image

def extract_name_spacy(text):

    nlp = spacy.load("en_core_web_lg")
    doc = nlp(text)

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            return ent.text.strip()
    return "Not Provided"
def extract_email(text):
    """Extracts email from resume text using regex"""
    match = re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", text)
    return match.group() if match else "Not Provided"
def extract_phone(text):
    """Extracts phone number from resume text using regex"""
    pattern = r"(\+?\d{1,3}[\s\-]?)?\(?\d{2,4}\)?[\s\-]?\d{3,4}[\s\-]?\d{3,4}"
    match = re.search(pattern, text)
    return match.group() if match else "Not Provided"


def extract_text_from_docx(file_path):
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    return "\n".join([page.extract_text() for page in reader.pages])

def preprocess_text(text):
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{4}', '', text)
    text = re.sub(r'(Resume Summary:.*?\n)', '', text)
    text = re.sub(r'[^\w\s.,-]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def generate_text(prompt):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    result = summarizer(prompt, max_length=300, min_length=100, do_sample=False)
    return result[0]["summary_text"]

def extract_section(lines, section_title):
    idx = next((i for i, ln in enumerate(lines) if section_title.lower() in ln.lower()), -1)
    if idx == -1:
        return "Not Provided"
    collected = []
    for line in lines[idx + 1:]:
        if any(word in line.upper() for word in ["EXPERIENCE", "SKILLS", "SUMMARY", "CERTIFICATIONS"]):
            break
        collected.append(line.strip())
    return " ".join(collected) if collected else "Not Provided"

def extract_technical_skills(lines):
    return extract_section(lines, "skills")

def extract_education(lines):
    return extract_section(lines, "education")

def extract_certifications(lines):
    return extract_section(lines, "certifications")

def extract_experience(lines):
    return extract_section(lines, "experience")

def extract_current_company(lines):
    for line in lines:
        if "present" in line.lower():
            return line.split("|")[0].strip()
    return "Not Provided"

def extract_total_experience(lines):
    return extract_section(lines, "total experience")

def extract_projects(lines):
    return extract_section(lines, "projects")

def automated_responder(content):
    cleaned_content = preprocess_text(content)
    prompt = f"""
Based on this resume content, write a brief professional summary in exactly this format:

[Name] is a [current status/role] with expertise in [key technical skills]. They hold [degree details] and have demonstrated proficiency in [specific tools/frameworks]. Their strongest areas are [top 2-3 domains].

Resume to analyze:
{cleaned_content[:2000]}

Important: Create a flowing, natural paragraph. Do not list raw resume content. Synthesize the information into a coherent summary."""
    summary = generate_text(prompt)
    return {"Professional Summary": summary}

def extract_resume_details(file_path):
    if file_path.lower().endswith(".docx"):
        extracted_text = extract_text_from_docx(file_path)
    elif file_path.lower().endswith(".pdf"):
        extracted_text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a .docx or .pdf file.")

    lines = extracted_text.split("\n")
    name = extract_name_spacy(extracted_text)
    email = extract_email(extracted_text)
    phone = extract_phone(extracted_text)
    candidate_id = generate_candidate_id(name, phone, email)
    summary = automated_responder(extracted_text)

    return {
        "candidate_id": candidate_id,
        "name": name,
        "email": email,
        "phone": phone,
        "summary": summary,
        "education": extract_education(lines),
        "skills": extract_technical_skills(lines),
        "certifications": extract_certifications(lines),
        "experience": extract_experience(lines),
        "current_company": extract_current_company(lines),
        "total_experience": extract_total_experience(lines),
        "projects": extract_projects(lines)
    }

if __name__ == "__main__":
    file_path = input("Enter the full path to the resume file (.docx or .pdf): ").strip()
    if not os.path.isfile(file_path):
        print("File does not exist.")
    else:
        try:
            details = extract_resume_details(file_path)
            output_file = "resume_details.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(details, f, indent=4, ensure_ascii=False)
            print("\nExtracted Resume Details:")
            print(json.dumps(details, indent=4, ensure_ascii=False))
            print(f"Saved to {output_file}")
        except Exception as e:
            print(f"Error: {e}")


Enter the full path to the resume file (.docx or .pdf): /content/resVishal.pdf


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Error: name 'generate_candidate_id' is not defined


**Details Parsing**

In [ ]:
import os
import re
import json
import hashlib
from datetime import datetime
from docx import Document
from PyPDF2 import PdfReader
import spacy
from transformers import pipeline

# Load NLP Model Once
nlp = spacy.load("en_core_web_lg")

# Load Summarization Model Once
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


def extract_name_spacy(text):
    """Extracts candidate name using spaCy NLP."""
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            return ent.text.strip()
    return "Not Provided"


def extract_email(text):
    """Extracts email from resume text using regex."""
    match = re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", text)
    return match.group() if match else "Not Provided"


def extract_phone(text):
    """Extracts phone number from resume text using regex."""
    pattern = r"(\+?\d{1,3}[\s\-]?)?\(?\d{2,4}\)?[\s\-]?\d{3,4}[\s\-]?\d{3,4}"
    match = re.search(pattern, text)
    return match.group() if match else "Not Provided"


def generate_candidate_id(name, phone, email):
    """Generates a unique candidate ID using MD5 hash."""
    raw_data = f"{name}{phone}{email}"
    return hashlib.md5(raw_data.encode()).hexdigest()


def extract_text_from_docx(file_path):
    """Extracts text from a DOCX file."""
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])


def extract_text_from_pdf(file_path):
    """Extracts text from a PDF file, handling potential NoneType errors."""
    reader = PdfReader(file_path)
    text = []
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text.append(page_text)
    return "\n".join(text)


def preprocess_text(text):
    """Cleans resume text by removing unwanted elements."""
    text = re.sub(r"\S+@\S+", "", text)  # Remove emails
    text = re.sub(r"\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{3,4}", "", text)  # Remove phone numbers
    text = re.sub(r"(Resume Summary:.*?\n)", "", text)  # Remove repeated phrases
    text = re.sub(r"[^\w\s.,-]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()  # Normalize whitespace
    return text


def generate_text(prompt):
    """Generates a professional summary using BART-Large-CNN model."""
    result = summarizer(prompt, max_length=300, min_length=100, do_sample=False)
    return result[0]["summary_text"]


def automated_responder(content):
    """Generates a professional summary based on extracted resume content."""
    cleaned_content = preprocess_text(content)

    prompt = f"""
Based on this resume content, write a brief professional summary in exactly this format:

[Name] is a [current status/role] with expertise in [key technical skills]. They hold [degree details] and have demonstrated proficiency in [specific tools/frameworks]. Their strongest areas are [top 2-3 domains].

Resume to analyze:
{cleaned_content[:2000]}

Important: Create a flowing, natural paragraph. Do not list raw resume content. Synthesize the information into a coherent summary."""

    summary = generate_text(prompt)

    return {"Professional Summary": summary}


def extract_section(lines, section_title):
    """Extracts specific sections from the resume."""
    idx = next((i for i, ln in enumerate(lines) if section_title.lower() in ln.lower()), -1)
    if idx == -1:
        return "Not Provided"
    collected = []
    for line in lines[idx + 1:]:
        if any(word in line.upper() for word in ["EXPERIENCE", "SKILLS", "SUMMARY", "CERTIFICATIONS"]):
            break
        collected.append(line.strip())
    return " ".join(collected) if collected else "Not Provided"


def extract_technical_skills(lines):
    return extract_section(lines, "skills")


def extract_education(lines):
    return extract_section(lines, "education")


def extract_certifications(lines):
    return extract_section(lines, "certifications")


def extract_experience(lines):
    return extract_section(lines, "experience")


def extract_current_company(lines):
    for line in lines:
        if "present" in line.lower():
            return line.split("|")[0].strip()
    return "Not Provided"


def extract_total_experience(lines):
    return extract_section(lines, "total experience")


def extract_projects(lines):
    return extract_section(lines, "projects")


def extract_resume_details(file_path):
    """Extracts structured resume details including name, email, phone, skills, education, experience, and projects."""
    if file_path.lower().endswith(".docx"):
        extracted_text = extract_text_from_docx(file_path)
    elif file_path.lower().endswith(".pdf"):
        extracted_text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a .docx or .pdf file.")

    lines = extracted_text.split("\n")
    name = extract_name_spacy(extracted_text)
    email = extract_email(extracted_text)
    phone = extract_phone(extracted_text)
    candidate_id = generate_candidate_id(name, phone, email)
    summary = automated_responder(extracted_text)

    return {
        "candidate_id": candidate_id,
        "name": name,
        "email": email,
        "phone": phone,
        "summary": summary,
        "education": extract_education(lines),
        "skills": extract_technical_skills(lines),
        "certifications": extract_certifications(lines),
        "experience": extract_experience(lines),
        "current_company": extract_current_company(lines),
        "total_experience": extract_total_experience(lines),
        "projects": extract_projects(lines)
    }


if __name__ == "__main__":
    file_path = input("Enter the full path to the resume file (.docx or .pdf): ").strip()
    if not os.path.isfile(file_path):
        print("File does not exist.")
    else:
        try:
            details = extract_resume_details(file_path)
            output_file = "resume_details.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(details, f, indent=4, ensure_ascii=False)
            print("\nExtracted Resume Details:")
            print(json.dumps(details, indent=4, ensure_ascii=False))
            print(f"Saved to {output_file}")
        except Exception as e:
            print(f"Error: {e}")


Device set to use cpu


Enter the full path to the resume file (.docx or .pdf): /content/resVishal.pdf

Extracted Resume Details:
{
    "candidate_id": "f8823d0911252075e0c7aa2017988c72",
    "name": " LinkedIn",
    "email": "vishalarvin.v@gmail.com",
    "phone": "Not Provided",
    "summary": {
        "Professional Summary": "Vishal Arvin is a graduate assistant at Florida Atlantic University. He has worked for Capgemini Engineering in Chennai, India. He developed interactive dashboards for Big Data visualization using Plotly. He is currently developing a custom chatbot that utilizes APIs and advance d natural language processing capabilities. For more information on Vishal's resume, visit his resume page here: http://www.vishalarvin.com/resume-page.html. For confidential support on suicide matters call the Samaritans on 08457 90 90 90 or visit a local Samaritans branch, see www.samaritans.org for details."
    },
    "education": "Florida Atlantic University   Dec 2024 Masters in, Computer Science   Cum

**Job Description**


In [ ]:
import os
import re
import logging
from typing import Dict, Tuple, Set
from docx import Document
from PyPDF2 import PdfReader
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class ResumeAnalyzer:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_lg")

    def extract_text(self, file_path: str) -> str:
        """Extract text from PDF or DOCX files"""
        if file_path.endswith('.pdf'):
            reader = PdfReader(file_path)
            text = []
            for page in reader.pages:
                text.append(page.extract_text())
            return "\n".join(text)
        elif file_path.endswith('.docx'):
            doc = Document(file_path)
            return "\n".join([para.text for para in doc.paragraphs])
        else:
            raise ValueError("Unsupported file format. Please provide PDF or DOCX file.")

    def preprocess_text(self, text: str) -> str:
        """Clean and preprocess text"""
        # Remove contact information
        text = re.sub(r'\S+@\S+', '', text)
        text = re.sub(r'\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{4}', '', text)

        # Remove special characters while preserving important punctuation
        text = re.sub(r'[^\w\s.,-:]', ' ', text)

        # Normalize whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        return text.lower()

    def extract_skills_from_job_description(self, text: str) -> Set[str]:
        """Extract skills from job description using NLP"""
        doc = self.nlp(text)
        skills = set()

        # Extract noun phrases as potential skills
        for chunk in doc.noun_chunks:
            # Clean the chunk text
            skill = chunk.text.lower().strip()

            # Filter out common non-skill phrases and very short terms
            if (len(skill.split()) <= 3 and  # Max 3 words
                len(skill) >= 3 and  # Min 3 characters
                not any(word.text.lower() in ['the', 'a', 'an', 'this', 'that', 'these', 'those']
                       for word in chunk) and
                not chunk.root.pos_ in ['PRON', 'DET', 'ADP']):  # Exclude pronouns, determiners, prepositions
                skills.add(skill)

        # Extract technical terms (proper nouns and acronyms)
        for token in doc:
            if (token.pos_ in ['PROPN'] or  # Proper nouns
                (token.text.isupper() and len(token.text) >= 2)):  # Acronyms
                skills.add(token.text.lower())

        # Add compound terms
        for token in doc:
            if token.dep_ == 'compound':
                compound = ' '.join([token.text, token.head.text]).lower()
                if len(compound.split()) <= 3:  # Max 3 words
                    skills.add(compound)

        return skills

    def find_matching_skills(self, text: str, required_skills: Set[str]) -> Set[str]:
        """Find matching skills in text"""
        doc = self.nlp(text)
        found_skills = set()

        # Convert text to lowercase for matching
        text_lower = text.lower()

        # Check for each required skill
        for skill in required_skills:
            # Look for exact matches
            if skill.lower() in text_lower:
                found_skills.add(skill)
                continue

            # Look for similar terms using spaCy similarity
            skill_doc = self.nlp(skill)
            for chunk in doc.noun_chunks:
                if skill_doc.similarity(chunk) > 0.85:  # High similarity threshold
                    found_skills.add(skill)
                    break

        return found_skills

    def analyze_resume(self, resume_text: str, job_description: str) -> Tuple[float, Set[str], Set[str], bool]:
        """Analyze resume against job description"""
        # Extract required skills from job description
        required_skills = self.extract_skills_from_job_description(job_description)

        # Find matching skills in resume
        matching_skills = self.find_matching_skills(resume_text, required_skills)

        # Calculate missing skills
        missing_skills = required_skills - matching_skills

        # Calculate similarity score
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform([resume_text, job_description])
        similarity_score = float(cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0])

        # Determine if resume is a good fit
        skill_match_ratio = len(matching_skills) / len(required_skills) if required_skills else 0
        is_fit = skill_match_ratio >= 0.7 and similarity_score > 0.6

        return similarity_score, matching_skills, missing_skills, is_fit

def main():
    analyzer = ResumeAnalyzer()

    # Get inputs
    resume_path = input("Enter the path to your resume file (PDF or DOCX): ").strip()
    job_description = input("Enter the job description: ").strip()

    try:
        # Extract and preprocess texts
        resume_text = analyzer.extract_text(resume_path)
        resume_text = analyzer.preprocess_text(resume_text)
        job_description = analyzer.preprocess_text(job_description)

        # Analyze resume
        similarity_score, matching_skills, missing_skills, is_fit = analyzer.analyze_resume(
            resume_text, job_description)

        # Print results
        print("\n=== Resume Analysis Results ===")

        print("\nMatching Skills:")
        for skill in sorted(matching_skills):
            print(f"  ✓ {skill}")

        print("\nMissing Skills:")
        for skill in sorted(missing_skills):
            print(f"  ✗ {skill}")

        print(f"\nSimilarity Score: {round(similarity_score * 100, 2)}%")
        print(f"Overall Assessment: {'Fit' if is_fit else 'Not Fit'} for the position")

        if not is_fit:
            print("\nRecommendation: Consider developing skills in the missing areas to better match the job requirements.")

    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Enter the path to your resume file (PDF or DOCX): /content/resVishal.pdf
Enter the job description: Posting Title: Software Developer - .NET and ReactJS The role of the Software Developer is to assist in providing technical and operational support to projects and programs. This role involves working under direct supervision to maintains, adapts and updates existing systems to meet user requirements and to enhance program efficiency. Acentra Health supports a high-volume healthcare data system that is accessed through multiple web portals.  What you will do: •	Maintains, adapts, and updates existing systems to meet user requirements and to enhance program efficiency. Researches and documents user requirements and system specifications. •	Designs and develops program logic addressing specific programming needs. Translates business requirements and functional specifications into detailed system and program design specifications. Assumes responsibility for program design activities includi

<ipython-input-14-8732cae4f72f>:95: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if skill_doc.similarity(chunk) > 0.85:  # High similarity threshold



=== Resume Analysis Results ===

Matching Skills:
  ✓ .net applications
  ✓ ability
  ✓ api
  ✓ asp.net api
  ✓ azure
  ✓ bachelor s
  ✓ business
  ✓ business requirements
  ✓ communication skills
  ✓ css
  ✓ data system
  ✓ demonstrable experience
  ✓ design specifications
  ✓ detailed system
  ✓ development lifecycle
  ✓ development tools
  ✓ ef
  ✓ existing systems
  ✓ experience
  ✓ functional specifications
  ✓ health
  ✓ healthcare data
  ✓ html5
  ✓ knowledge
  ✓ lifecycle documentation
  ✓ maintenance
  ✓ maintenance environment
  ✓ management
  ✓ multiple web portals
  ✓ operation
  ✓ operational maintenance environment
  ✓ practical problems
  ✓ projects
  ✓ relational database concepts
  ✓ reports
  ✓ software development
  ✓ solving skills
  ✓ sql
  ✓ strong interpersonal skills
  ✓ system designs
  ✓ system level
  ✓ system software
  ✓ system specifications
  ✓ systems
  ✓ systems development
  ✓ systems development lifecycle
  ✓ t
  ✓ technical requirements
  ✓ techniqu

In [ ]:
import os
import re
import hashlib
import json

# For DOCX
from docx import Document

# For PDF (PyPDF2)
from PyPDF2 import PdfReader

# For SpaCy
import spacy

# ---------------------------------------------------------
#                  File Reading Functions
# ---------------------------------------------------------
def extract_text_from_docx(file_path):
    """
    Extract non-empty paragraphs from a DOCX file.
    """
    try:
        doc = Document(file_path)
        return [para.text.strip() for para in doc.paragraphs if para.text.strip()]
    except Exception as e:
        raise ValueError(f"Error reading DOCX file: {str(e)}")

def extract_text_from_pdf(file_path):
    """
    Extract text from all pages of a PDF file using PyPDF2.
    """
    try:
        reader = PdfReader(file_path)
        text_lines = []
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text_lines.extend(page_text.strip().split("\n"))
        return [line.strip() for line in text_lines if line.strip()]
    except Exception as e:
        raise ValueError(f"Error reading PDF file: {str(e)}")

# ---------------------------------------------------------
#               Heading Identification
# ---------------------------------------------------------
def find_headings(text_lines):
    """
    Identify 'headings' in the lines based on capitalization or short lines.
    This is a basic heuristic you can refine as needed.
    """
    headings = []
    for line in text_lines:
        # Potential conditions for a heading:
        #   1) Entire line is uppercase (line.isupper())
        #   2) A short line (<= 4 words) possibly with trailing punctuation
        #   3) Lines matching "SOMETHING:"
        if line.isupper() or re.match(r"^\s*[A-Za-z]+\s*(\:|\-)$", line) or len(line.split()) <= 4:
            headings.append(line.strip())
    return headings

def parse_content_by_headings(text_lines):
    """
    Group lines under identified headings.
    Returns a dict: { heading: "content under that heading" }
    """
    headings = find_headings(text_lines)
    content = {}
    current_heading = None

    for line in text_lines:
        if line in headings:
            current_heading = line
            content[current_heading] = []
        elif current_heading:
            content[current_heading].append(line)

    # Convert lists to single string
    for key in content:
        content[key] = " ".join(content[key]).strip()

    return content

# ---------------------------------------------------------
#           Named Entity Recognition (SpaCy)
# ---------------------------------------------------------
# Load the SpaCy model (make sure you've installed it, e.g.:
# `python -m spacy download en_core_web_sm`)
nlp = spacy.load("en_core_web_sm")

# ---------------------------------------------------------
#      Candidate ID Helper
# ---------------------------------------------------------
def generate_candidate_id(name, phone, email):
    """
    Create a unique MD5 hash based on name, phone, and email.
    """
    id_source = f"{name}{phone}{email}"
    candidate_id = hashlib.md5(id_source.encode()).hexdigest()
    return candidate_id

# ---------------------------------------------------------
#        Main Extraction (Returns Desired Dictionary)
# ---------------------------------------------------------
def extract_resume_details(file_path):
    """
    1. Read text from DOCX or PDF
    2. Identify headings and parse content
    3. Extract name, email, phone
    4. Construct a final dict with all required keys
    """
    # -------------------------------
    # 1. Extract text lines
    # -------------------------------
    if file_path.lower().endswith('.docx'):
        text_lines = extract_text_from_docx(file_path)
    elif file_path.lower().endswith('.pdf'):
        text_lines = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please use PDF or DOCX.")

    full_text = "\n".join(text_lines)

    # -------------------------------
    # 2. Parse headings
    #    (Optional, for analysis or extraction)
    # -------------------------------
    headings_dict = parse_content_by_headings(text_lines)
    # You could refine logic to glean "skills", "education", etc. from headings_dict

    # -------------------------------
    # 3. Basic info extraction
    # -------------------------------
    doc = nlp(full_text)

    # Name: first PERSON entity, else "Unknown"
    name = next((ent.text for ent in doc.ents if ent.label_ == "PERSON"), "Unknown")

    # Email: simple regex
    email_match = re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", full_text)
    email = email_match.group() if email_match else "No Information"

    # Phone: simple 10-digit pattern (adjust to your local formatting)
    phone_match = re.search(r"\b\d{10}\b", full_text)
    phone = phone_match.group() if phone_match else "No Information"

    # Candidate ID
    candidate_id = generate_candidate_id(name, phone, email)

    # -------------------------------
    # 4. Example placeholders or naive extractions
    #    (For real usage, parse the headings or use advanced parsing.)
    # -------------------------------
    # "Skills" - naive approach: scan for known keywords
    skills_pattern = r"(Python|Java|SQL|AWS|Docker|Machine Learning)"
    found_skills = re.findall(skills_pattern, full_text, re.IGNORECASE)
    # Deduplicate & join
    skills = ", ".join(set(s.capitalize() for s in found_skills)) if found_skills else "Not Provided"

    # "Education" - naive approach: look for keywords in the text
    if "education" in headings_dict:
        education = headings_dict["education"]
    else:
        # or just do a naive search for "Bachelor", "Master", etc.
        edu_match = re.search(r"(Bachelor|Master|B\.Sc|M\.Sc|PhD|University|College)", full_text, re.IGNORECASE)
        education = edu_match.group() if edu_match else "Not Provided"

    # "Certifications"
    # You might parse from headings_dict or do a naive approach:
    cert_match = re.search(r"(certificate|certification|trained)", full_text, re.IGNORECASE)
    certifications = cert_match.group() if cert_match else "Not Provided"

    # "Experience" - naive approach
    # Possibly you want content under a heading like "EXPERIENCE"
    experience = headings_dict.get("EXPERIENCE", "Not Provided")

    # "Current Company" - naive approach
    current_company = "Not Provided"

    # "Total Experience" - naive approach
    total_experience = "Not Provided"

    # "Summary" - naive approach:
    summary = headings_dict.get("SUMMARY", "Not Provided")

    # "Courses"
    courses = "Not Provided"

    # -------------------------------
    # 5. Construct final dictionary
    # -------------------------------
    output = {
        "candidate_id": candidate_id,
        "name": name,
        "email": email,
        "phone": phone,
        "skills": skills,
        "education": education,
        "certifications": certifications,
        "experience": experience,
        "current_company": current_company,
        "total_experience": total_experience,
        "summary": summary,
        "courses": courses
    }

    return output


# ---------------------------------------------------------
#                Example Main Usage
# ---------------------------------------------------------
if __name__ == "__main__":
    file_path = input("Enter the full path to the resume file (PDF or DOCX): ")

    if not os.path.exists(file_path):
        print("File does not exist.")
    else:
        try:
            details = extract_resume_details(file_path)

            # Save to JSON
            output_file = "resume_details.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(details, f, indent=4, ensure_ascii=False)

            print("\nExtracted Resume Details (Sample):")
            print(json.dumps(details, indent=4, ensure_ascii=False))
            print(f"Extracted resume details saved to {output_file}")

        except Exception as e:
            print(f"Error: {e}")


Enter the full path to the resume file (PDF or DOCX): /content/resVishal.pdf

Extracted Resume Details (Sample):
{
    "candidate_id": "6152c390e3e0dbc51ae2a15c3b9125a6",
    "name": "V Vishal Arvin",
    "email": "vishalarvin.v@gmail.com",
    "phone": "No Information",
    "skills": "Docker, Machine learning, Python, Aws, Java, Sql",
    "education": "University",
    "certifications": "CERTIFICATION",
    "experience": "Graduate Assistant  Jan 2024 – present Florida Atlantic University | Boca Raton , USA • Engineered and implemented automation scripts using Python and shell scripting to streamline the student application process, significantly reducing processing time and enhancing operational efficiency . • Developed and deployed interactive dashboards for Big Data visualization using Plotly, enabling efficient data analysis and r eporting",
    "current_company": "Not Provided",
    "total_experience": "Not Provided",
    "summary": "Not Provided",
    "courses": "Not Provided"
}


In [ ]:
from transformers import pipeline
import docx
from docx import Document
from PyPDF2 import PdfReader
import os
import re
import json
import hashlib
from datetime import datetime
from docx import Document
from PyPDF2 import PdfReader
import spacy
from transformers import pipeline
from PIL import Image
# Function to extract text from a .docx file
def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

# Function to extract text from a .pdf file
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    return "\n".join([page.extract_text() for page in reader.pages])

# Function to generate text using Hugging Face Transformers
def generate_text(prompt):
    text_generator = pipeline("text-generation", model="gpt2")
    result = text_generator(prompt, max_length=600, num_return_sequences=1,truncation=True)
    return result[0]["generated_text"]


# Automated responder based on extracted sections
def automated_responder(content):
    # Predefined prompts for generating response sections
    prompts = {
        "General Information": f"Extract the basic information from the RFP, including the issuing organization, proposal due date, contact details, and submission requirements\n\n{content[:500]}",
        "Scope of Work": f"Extract the scope of work or project requirements outlined in the RFP, specifying deliverables, objectives, and key milestones\n\n{content[:500]}",
        "Evaluation Criteria": f"Identify and extract the evaluation criteria mentioned in the RFP, including scoring or weighting factors if provided:\n\n{content[:500]}",
        "Proposal Submission Instructions": f"Extract detailed instructions for proposal submission, including format, required documents, and submission methods (email, portal, hard copy).\n\n{content[:500]}",
    }

    # Generate responses for each section
    responses = {}
    for section, prompt in prompts.items():
        responses[section] = generate_text(prompt)

    return responses

# Function to save responses to a .docx template
def save_to_docx(responses, output_path):
    doc = Document()
    doc.add_heading("Project Proposal", level=1)

    for section, content in responses.items():
        doc.add_heading(section, level=2)
        doc.add_paragraph(content)

    doc.save(output_path)

# Main testing function
def main(file_path, output_path):
    if file_path.endswith(".docx"):
        extracted_text = extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        extracted_text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Use a .docx or .pdf file.")

    #print("Extracted Text:")
    #print(extracted_text)

    # Generate text based on the extracted content
    #print("\nGenerated Text:")
    #print(generate_text(extracted_text[:500]))  # Limit prompt length for GPT-2

    # Generate automated response
    print("\nGenerated Automated Response:")
    responses = automated_responder(extracted_text)
    for section, response in responses.items():
        print(f"\n{section}:\n{response}")

    # Save the responses to a .docx template
    save_to_docx(responses, output_path)
    print(f"\nProposal saved to {output_path}")

if __name__ == "__main__":
    file_path = "/content/resVishal.pdf"  # Replace with your .docx or .pdf file path
    #"C:\Users\tejas\Downloads\23_0481-pub-RFP-On-Demand_Information_Technology_Proje\13-Specification-230481.pdf"
    output_path = "/content/sample_data/output.docx"
    main(file_path, output_path)

In [ ]:
pip install langchain-community ollama python-docx PyPDF2 pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15


In [ ]:
from transformers import pipeline
import docx
from docx import Document
from PyPDF2 import PdfReader
import re

# Function to extract text from a .docx file
def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

# Function to extract text from a .pdf file
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    return "\n".join([page.extract_text() for page in reader.pages])

# Function to preprocess the text (remove noise and repetition)
def preprocess_text(text):
    # Remove email addresses and phone numbers
    text = re.sub(r'\S+@\S+', '', text)  # Remove emails
    text = re.sub(r'\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{4}', '', text)  # Remove phone numbers
    # Remove repeated phrases (e.g., "Resume Summary")
    text = re.sub(r'(Resume Summary:.*?\n)', '', text)
    # Remove special characters and extra whitespace
    text = re.sub(r'[^\w\s.,-]', '', text)  # Keep only alphanumeric, spaces, commas, periods, and hyphens
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Function to generate text using BART for summarization
def generate_text(prompt):
    # Load BART model for summarization
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    # Generate a concise summary
    result = summarizer(prompt, max_length=300, min_length=100, do_sample=False)
    return result[0]["summary_text"]

# Automated responder based on extracted sections
'''def automated_responder(content):
    # Preprocess the input text to remove noise
    cleaned_content = preprocess_text(content)
    # Use the provided prompt for generating a comprehensive summary
    prompt = (
        "Summarize the following resume for a hiring manager. Include:\n"
        "- A professional overview of the candidate's expertise and career highlights.\n"
        "- Key technical and soft skills.\n"
        "- Relevant roles, contributions, and outcomes.\n"
        "- Education and certifications.\n"
        "- Notable projects or accomplishments.\n\n"
        f"{cleaned_content[:2000]}"  # Use the first 2000 characters of the cleaned resume text as input
    )'''
def automated_responder(content):
    # Preprocess the input text to remove noise
    cleaned_content = preprocess_text(content)

    # More directive prompt that forces synthesis
    prompt = f"""
Based on this resume content, write a brief professional summary in exactly this format:

[Name] is a [current status/role] with expertise in [key technical skills]. They hold [degree details] and have demonstrated proficiency in [specific tools/frameworks]. Their strongest areas are [top 2-3 domains].

Resume to analyze:
{cleaned_content[:2000]}

Important: Create a flowing, natural paragraph. Do not list raw resume content. Synthesize the information into a coherent summary."""

    # Generate the summary
    summary = generate_text(prompt)

    return {
        "Professional Summary": summary
    }
    # Generate the summary
    summary = generate_text(prompt)

    return {
        "Executive Summary": summary
    }

    # Generate the summary
    summary = generate_text(prompt)
    return {"Comprehensive Summary": summary}

# Function to save responses to a .docx template
def save_to_docx(responses, output_path):
    doc = Document()
    doc.add_heading("Resume Summary for Hiring Manager", level=1)

    for section, content in responses.items():
        doc.add_heading(section, level=2)
        doc.add_paragraph(content)

    doc.save(output_path)

# Main testing function
def main(file_path, output_path):
    if file_path.endswith(".docx"):
        extracted_text = extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        extracted_text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Use a .docx or .pdf file.")

    # Generate automated response
    print("\nGenerated Automated Response:")
    responses = automated_responder(extracted_text)
    for section, response in responses.items():
        print(f"\n{section}:\n{response}")

    # Save the responses to a .docx template
    save_to_docx(responses, output_path)
    print(f"\nSummary saved to {output_path}")

if __name__ == "__main__":
    file_path = "/content/Anita_Kamma_2517 (1).docx"  # Replace with your .docx or .pdf file path
    output_path = "/content/sample_data/output.docx"
    main(file_path, output_path)


Generated Automated Response:


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



Professional Summary:
Anita Rani Kamma has 9 years of experience in developing, implementing Web Applications, Windows Applications and products using Visual Studio 2022201920172015201320102008, c.NET, ASP.NET,. Team Foundation Server TFS, Dot NET Framework, Azure DevOps and Databases like SQL Server 201220082005 with N-tier development environment. Extensive working experience with different methodologies like Agile Scrum and Waterfall model. Experience in using various test cases and performing unit testing and integration testing.

Summary saved to /content/sample_data/output.docx


In [ ]:
pip install PyMuPDF opencv-python pytesseract pillow transformers torch sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 834.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:

!pip install pymupdf
!pip install sumy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 58.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.9 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=d20f33b5c7b3bbb7e5e1a4f1cd4bcccf603c1a4750a1bae408dbc07fcdba8071
  Stored in directory: /root/.cache/pip/wheels/4d/57/58/7e3d7fedf51fe248b7fcee3df6945ae28638e22cddf01eb92b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=5beb66aae24b17bc9f832ddd909a4b14b2de317ac521a257a8243ca2f377bb1e
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built breadability docopt


#Summary Extraction

In [ ]:
import fitz  # PyMuPDF
import nltk
import re
import json
from typing import Dict, List, Optional
from pathlib import Path
from transformers import pipeline
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# Ensure necessary NLTK data is available
nltk.download('punkt', quiet=True)

class ResumeSummarizer:
    """Class to extract and summarize key sections from a resume."""

    def __init__(self, model_name: str = "facebook/bart-large-cnn"):
        print("Loading BART model for summarization...")
        self.summarizer = pipeline("summarization", model=model_name)
        self.sumy_summarizer = LsaSummarizer()
        print("Summarizer initialized successfully.")

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """Extract text from a PDF resume."""
        print(f"\nExtracting text from: {pdf_path}")
        pdf_path = Path(pdf_path)
        if not pdf_path.exists():
            raise FileNotFoundError(f"Error: PDF file not found: {pdf_path}")

        with fitz.open(pdf_path) as doc:
            text = "\n".join([page.get_text("text") for page in doc])
        return text.strip()

    def parse_resume_sections(self, text: str) -> Dict[str, str]:
        """Identify and extract structured sections from a resume."""
        sections = {
            "📌 Contact Information": [],
            "📌 Summary": [],
            "📌 Experience": [],
            "📌 Education": [],
            "📌 Skills": [],
            "📌 Projects": [],
            "📌 Certifications": [],
            "📌 Other Information": []
        }

        # Define section headers to look for
        section_headers = {
            "📌 Contact Information": ["contact", "phone", "email", "linkedin", "address"],
            "📌 Summary": ["summary", "objective"],
            "📌 Experience": ["experience", "work history", "employment", "career"],
            "📌 Education": ["education", "academic background", "studies"],
            "📌 Skills": ["skills", "technical skills", "competencies"],
            "📌 Projects": ["projects", "portfolio"],
            "📌 Certifications": ["certifications", "licenses"],
            "📌 Other Information": ["volunteer", "languages", "interests", "additional information"]
        }

        # Normalize text
        lines = text.split("\n")
        current_section = "📌 Other Information"  # Default section

        for line in lines:
            line_lower = line.lower().strip()

            # Check if the line is a known section header
            for section, keywords in section_headers.items():
                if any(keyword in line_lower for keyword in keywords):
                    current_section = section
                    break

            # Add line to the current section
            if line.strip():
                sections[current_section].append(line.strip())

        # Convert lists to strings
        return {section: "\n".join(content).strip() for section, content in sections.items() if content}

    def summarize_section(self, text: str, max_length: int = 150) -> str:
        """Summarize a given text section."""
        if not text:
            return "No relevant information found."

        try:
            summary = self.summarizer(
                text,
                max_length=max_length,
                min_length=50,
                do_sample=False
            )[0]['summary_text']
            return summary
        except Exception as e:
            print(f"Error summarizing section: {e}")
            return text[:max_length] + "..."

    def process_resume(self, pdf_path: str) -> Dict[str, str]:
        """Process the resume and generate structured summaries."""
        extracted_text = self.extract_text_from_pdf(pdf_path)
        parsed_sections = self.parse_resume_sections(extracted_text)

        results = {}
        for section, content in parsed_sections.items():
            print(f"\nProcessing section: {section}")
            summarized_content = self.summarize_section(content)
            results[section] = summarized_content

        # Save results as JSON
        output_path = Path(pdf_path).with_suffix('.json')
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        print(f"\nResults saved to: {output_path}")

        # Print the structured summary
        print("\n=== Structured Summary ===")
        for section, content in results.items():
            print(f"\n{section}:\n{content}")

        return results


def main():
    """Main function to run the resume summarizer."""
    print("\n=== Resume Summarizer ===")
    summarizer = ResumeSummarizer()

    while True:
        pdf_path = input("\nEnter the path to the resume PDF: ").strip()

        if not pdf_path or not Path(pdf_path).exists():
            print("Invalid path. Please enter a valid resume PDF file.")
            continue

        results = summarizer.process_resume(pdf_path)

        another = input("\nWould you like to process another resume? (yes/no): ").lower().strip()
        if another in ['no', 'n']:
            print("Thank you for using the Resume Summarizer!")
            break


if __name__ == "__main__":
    main()


KeyboardInterrupt: 

# se-2

In [ ]:
import re
import os
import logging
import fitz  # PyMuPDF
import pytesseract
import torch
import cv2
import numpy as np
from PIL import Image
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk

# Ensure NLTK sentence tokenizer is available
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def preprocess_image(image):
    """Convert image to grayscale, apply adaptive thresholding, and denoise for better OCR."""
    if isinstance(image, np.ndarray):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = np.array(image.convert('L'))
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )
    denoised = cv2.fastNlMeansDenoising(thresh)
    return denoised

def extract_text_from_file(file_path):
    """
    Extract text from PDFs or images.
    Uses PyMuPDF for PDFs and falls back to OCR (via pytesseract) if direct extraction returns little text.
    """
    ext = os.path.splitext(file_path)[1].lower()
    full_text = ""
    if ext == ".pdf":
        try:
            doc = fitz.open(file_path)
            for page in doc:
                text = page.get_text()
                # If the extracted text is very short, assume this page is scanned.
                if not text.strip() or len(text.strip()) < 20:
                    pix = page.get_pixmap()
                    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                    processed_img = preprocess_image(img)
                    text = pytesseract.image_to_string(Image.fromarray(processed_img))
                full_text += f"\n--- PAGE ---\n{text}"
            return full_text
        except Exception as e:
            logger.error(f"Error extracting PDF: {e}")
            return ""
    elif ext in ['.png', '.jpg', '.jpeg', '.tiff', '.bmp']:
        try:
            img = Image.open(file_path)
            processed_img = preprocess_image(img)
            text = pytesseract.image_to_string(Image.fromarray(processed_img))
            return text
        except Exception as e:
            logger.error(f"Error extracting image: {e}")
            return ""
    else:
        logger.error("Unsupported file type")
        return ""

def preprocess_text(text):
    """Clean the text by normalizing whitespace and newlines."""
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n+', '\n', text)
    return text.strip()

def extract_section(text, section_name):
    """
    Attempt to extract the content of a section using multiple regex patterns.
    Returns the text found or an empty string if not found.
    """
    patterns = [
        rf"(?i){re.escape(section_name)}[:\s\-]*(.*?)(?=\n[A-Z])",
        rf"(?i){re.escape(section_name)}\s*\n(.*?)(?=\n[A-Z])"
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(1).strip()
    return ""

def extract_relevant_content(section_heading, section_text, target_word_count=50):
    """
    Split the section text into sentences and select those that share words with the section heading.
    Then combine and truncate the result to the target word count.
    """
    sentences = nltk.sent_tokenize(section_text)
    heading_words = set(section_heading.lower().split())
    relevant = [sent for sent in sentences if heading_words.intersection(set(sent.lower().split()))]
    if not relevant and sentences:
        relevant = [sentences[0]]
    combined = " ".join(relevant)
    words = combined.split()
    if len(words) > target_word_count:
        return " ".join(words[:target_word_count]) + "..."
    return combined

def summarize_text_custom(text, section_heading, tokenizer, model, max_length, min_length):
    """
    Summarize the text using the transformer model.
    A custom prompt is prepended instructing the model to focus on key points.
    If the input is too short for summarization, the original text is returned.
    """
    prompt = (f"Summarize the following '{section_heading}' section from an RFP document. "
              "Focus on key requirements, deliverables, objectives, and essential details. "
              f"Provide a clear, concise summary in approximately {max_length} tokens.\n\n{text}")
    tokenized = tokenizer.tokenize(prompt)
    # If the prompt is very short, skip summarization.
    if len(tokenized) < min_length:
        return text
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(input_ids,
                                     max_length=max_length,
                                     min_length=min_length,
                                     length_penalty=2.0,
                                     num_beams=4,
                                     early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary
    except Exception as e:
        logger.error(f"Summarization error: {e}")
        return text[:max_length]

def process_sections(full_text, target_sections, tokenizer, model):
    """
    Process each target section:
      - For 'Scope of Work' and 'Project Background', use the summarizer to get a summary (around 200 tokens).
      - For all other sections, extract sentences that are relevant to the section heading.
    Returns a dictionary with both the raw extraction and the final summary.
    """
    results = {}
    for section, desired_length in target_sections.items():
        sec_text = extract_section(full_text, section)
        if not sec_text:
            results[section] = {"extracted": "", "summary": "Section not found."}
            continue
        if section.lower() in ["scope of work", "project background"]:
            # Use the custom summarization function.
            summary = summarize_text_custom(sec_text, section, tokenizer, model,
                                            max_length=desired_length + 50,  # Allow extra tokens
                                            min_length=desired_length)
        else:
            # For other sections, extract only sentences relevant to the section heading.
            summary = extract_relevant_content(section, sec_text, target_word_count=desired_length)
        results[section] = {"extracted": sec_text, "summary": summary}
    return results

def main():
    # IMPORTANT: Update this with the path to your document
    pdf_file = "/content/STAFF-8601 (2).pdf"
    if not os.path.exists(pdf_file):
        print(f"ERROR: File not found at {pdf_file}")
        return

    # Extract and clean the text from the file.
    full_text = extract_text_from_file(pdf_file)
    full_text = preprocess_text(full_text)

    # Define the sections with desired summary lengths (for summarization tokens or target word counts).
    # For summarization sections, desired_length is used as the approximate minimum token count.
    target_sections = {
        "scope of work": 200,         # Summarize to ~200 tokens (can be adjusted)
        "project background": 200,    # Summarize to ~200 tokens
        "deliverables": 50,           # Extract 50 relevant words
        "timeline": 50,               # Extract 50 relevant words
        "budget": 50,                 # Extract 50 relevant words
        "pricing": 50                 # Extract 50 relevant words
    }

    # Load the summarization model and tokenizer.
    # You may consider using a larger model (e.g. "facebook/bart-large-cnn") for improved performance.
    model_name = "sshleifer/distilbart-cnn-12-6"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    section_results = process_sections(full_text, target_sections, tokenizer, model)

    # Print the final summaries.
    print("\n--- RFP SECTION ANALYSIS ---\n")
    for section, data in section_results.items():
        print(f"--- {section.upper()} ---")
        print(data["summary"])
        print("-" * 50)

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]


--- RFP SECTION ANALYSIS ---

--- SCOPE OF WORK ---
Section not found.
--------------------------------------------------
--- PROJECT BACKGROUND ---
Section not found.
--------------------------------------------------
--- DELIVERABLES ---
Section not found.
--------------------------------------------------
--- TIMELINE ---
Section not found.
--------------------------------------------------
--- BUDGET ---
Section not found.
--------------------------------------------------
--- PRICING ---
Section not found.
--------------------------------------------------
